# 행정동별 인구 데이터 정제 코드

---

- 최종 수정일 : 2024-03-19
- chapter 6.4에서 샘플 데이터와 병합하는 데이터를 정제하는 코드입니다.

In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_excel('census_data.xlsx')
df['총인구수'] = df['총인구수'].apply(lambda x: int(x.replace(",", "")))
df['세대수'] = df['세대수'].apply(lambda x: int(x.replace(",", "")))

### 1. 광역자치단체 단위로 합산된 경우 제외

`drop_data` 
- 광역자치단체 단위로 합산된 데이터를 제외합니다.

In [19]:
def drop_data(x):
    if len(x.split(" ")) < 3 and x.split(" ")[0] != "세종특별자치시":
        return False
    else:
        if '' not in x.split(" "):
        
            return True
        else:
            return False


In [3]:
df["drop"] = df['행정기관'].apply(lambda x: drop_data(x)) # 삭제할 데이터인지 구분하는 컬럼을 추가합니다.
df.drop(df[df['drop'] == False].index, axis=0, inplace=True) # 해당 컬럼에서 False인 값만 삭제합니다.
df.drop('drop', axis=1, inplace=True) # 임시로 생성했던 컬럼을 삭제합니다
df.reset_index(inplace=True, drop=True)

### 2. 출장소 데이터 제외 및 세부 행정동 통합
- 출장소 단위는 제외합니다.
- '종로1가동'과 같이 하나의 법정동이 다수의 행정동으로 나뉜 경우, 하나의 법정동으로 통합합니다.

In [13]:
def local_name_cleaner(x):
    numerics = re.search(r'(?:제*)\d(.[\d])*+', x)
    office=re.search(r'(?P<local>[가-힣]+(?:읍|면|동|구|시|도))+(?P<office>[가-힣]+출장소)', x)
    subnum = re.search(r"\d가", x)
    newadd = x
    if numerics:
        drop_idx = numerics.span()
        newadd = x.replace(x[drop_idx[0]:drop_idx[1]], "")
    else:
        pass
    
    if office:
        drop_word = office["office"]
        newadd = newadd.replace(drop_word, "")
    else:
        pass
    
    if subnum:
        drop_word = subnum.span()
        newadd = newadd.replace(newadd[drop_word[0]:drop_word[1]], "")
    
    return newadd

df['행정기관'] = df['행정기관'].apply(lambda x: local_name_cleaner(x))
df

,행정기관,총인구수,세대수,세대당 인구,남여 비율,동명칭
0,강원특별자치도 강릉시 강남동,16890,8081,2.09,0.99,강남동
1,강원특별자치도 강릉시 강동면,3912,2188,1.79,1.12,강동면
2,강원특별자치도 강릉시 경포동,11263,5218,2.16,1.05,경포동
3,강원특별자치도 강릉시 교동,32352,15523,4.08,1.93,교동
4,강원특별자치도 강릉시 구정면,4178,2032,2.06,1.03,구정면
...,...,...,...,...,...,...
2843,충청북도 충주시 주덕읍,4737,2660,1.78,1.07,주덕읍
2844,충청북도 충주시 중앙탑면,14745,6510,2.26,1.06,중앙탑면
2845,충청북도 충주시 지현동,4988,2477,2.01,0.99,지현동
2846,충청북도 충주시 칠금.금릉동,13951,6283,2.22,0.97,칠금.금릉동


In [14]:
# 세부 헹정동으로 나뉜 경우를 모두 합쳐줍니다
df = df.groupby("행정기관").sum()
df.reset_index(inplace=True)
df

C:\Users\user\AppData\Local\Temp\ipykernel_21244\3448576034.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby("행정기관").sum()


,총인구수,세대수,세대당 인구,남여 비율
행정기관,,,,
강원특별자치도 강릉시 강남동,16890,8081,2.09,0.99
강원특별자치도 강릉시 강동면,3912,2188,1.79,1.12
강원특별자치도 강릉시 경포동,11263,5218,2.16,1.05
강원특별자치도 강릉시 교동,32352,15523,4.08,1.93
강원특별자치도 강릉시 구정면,4178,2032,2.06,1.03
...,...,...,...,...
충청북도 충주시 주덕읍,4737,2660,1.78,1.07
충청북도 충주시 중앙탑면,14745,6510,2.26,1.06
충청북도 충주시 지현동,4988,2477,2.01,0.99


### 4. 동명칭 컬럼 추가

In [17]:
df['동명칭'] = df['행정기관'].apply(lambda x: x.split(" ")[-1])
df

,행정기관,총인구수,세대수,세대당 인구,남여 비율,동명칭
0,강원특별자치도 강릉시 강남동,16890,8081,2.09,0.99,강남동
1,강원특별자치도 강릉시 강동면,3912,2188,1.79,1.12,강동면
2,강원특별자치도 강릉시 경포동,11263,5218,2.16,1.05,경포동
3,강원특별자치도 강릉시 교동,32352,15523,4.08,1.93,교동
4,강원특별자치도 강릉시 구정면,4178,2032,2.06,1.03,구정면
...,...,...,...,...,...,...
2839,충청북도 충주시 주덕읍,4737,2660,1.78,1.07,주덕읍
2840,충청북도 충주시 중앙탑면,14745,6510,2.26,1.06,중앙탑면
2841,충청북도 충주시 지현동,4988,2477,2.01,0.99,지현동
2842,충청북도 충주시 칠금.금릉동,13951,6283,2.22,0.97,칠금.금릉동


In [18]:
df.to_excel('census_data_cleaned.xlsx', index=False)

### 5. 합산된 행정동 확인하기
- 인구가 적은 법정동의 경우 다수의 2개 이상의 법정동으로 합산하여 인구 통계를 냅니다.
- 이를 반영하여, 샘플데이터에서 생성되는 키값을 조정하는 테이블을 생성하였습니다.

In [161]:
def find(x):
    if "." in x:
        return True
    else:
        return False
df['dotin'] = df['동명칭'].apply(lambda x: find(x) )

In [162]:
df[df['dotin'] == True]

,행정기관,총인구수,세대수,세대당 인구,남여 비율,동명칭,dotin
1354,대구광역시 동구 불로.봉무동,20294,8606,2.36,0.97,불로.봉무동,True
1893,인천광역시 동구 화수.화평동,5518,2779,1.99,1.04,화수.화평동,True
2794,충청북도 청주시 상당구 용담.명암.산성동,12867,4832,2.66,0.97,용담.명암.산성동,True
2804,충청북도 청주시 서원구 성화.개신.죽림동,42415,20067,2.11,0.98,성화.개신.죽림동,True
2813,충청북도 청주시 청원구 율량.사천동,48156,22176,2.17,1.00,율량.사천동,True
2818,충청북도 청주시 흥덕구 봉명.송정동,24029,12810,1.88,1.17,봉명.송정동,True
2822,충청북도 청주시 흥덕구 운천.신봉동,14482,6961,2.08,0.96,운천.신봉동,True
2823,충청북도 충주시 교현.안림동,22544,10130,2.23,0.98,교현.안림동,True
2830,충청북도 충주시 목행.용탄동,6384,2843,2.25,1.11,목행.용탄동,True
2835,충청북도 충주시 성내.충인동,2484,1453,1.71,0.96,성내.충인동,True


In [ ]:
transform = {
    "data" : [
        {
             "불로동" : {
            "city" : "대구광역시 동구",
            "changeto" : "불로.봉무동"
             }
        },
        {
             "봉무동" : {
            "city" : "대구광역시 동구",
            "changeto" : "불로.봉무동"
             }
        },
        {
             "화수동" : {
            "city" : "인천광역시 동구",
            "changeto" : "화수.화평동"
             }
        },
        {
             "화평동" : {
            "city" : "인천광역시 동구",
            "changeto" : "화수.화평동"
             }
        },
        {
             "용담동" : {
            "city" : "충청북도 청주시",
            "changeto" : "용담.명암.산성동"
             }
        },
        {
             "명암동" : {
            "city" : "충청북도 청주시",
            "changeto" : "용담.명암.산성동"
             }
        },
        {
             "산성동" : {
            "city" : "충청북도 청주시",
            "changeto" : "용담.명암.산성동"
             }
        },
        {
             "성화동" : {
            "city" : "충청북도 청주시",
            "changeto" : "성화.개신.죽림동"
             }
        },
        {
             "개신동" : {
            "city" : "충청북도 청주시",
            "changeto" : "성화.개신.죽림동"
             }
        },
        {
             "죽림동" : {
            "city" : "충청북도 청주시",
            "changeto" : "성화.개신.죽림동"
             }
        },
        {
             "율량동" : {
            "city" : "충청북도 청주시",
            "changeto" : "율량.사천동"
             }
        },
        {
             "사천동" : {
            "city" : "충청북도 청주시",
            "changeto" : "율량.사천동"
             }
        },
        {
             "봉명동" : {
            "city" : "충청북도 청주시",
            "changeto" : "봉명.송정동"
             }
        },
        {
             "송정동" : {
            "city" : "충청북도 청주시",
            "changeto" : "봉명.송정동"
             }
        },
        {
             "운천동" : {
            "city" : "충청북도 청주시",
            "changeto" : "운천.신봉동"
             }
        },
        {
             "신봉동" : {
            "city" : "충청북도 청주시",
            "changeto" : "운천.신봉동"
             }
        },
        {
             "교현동" : {
            "city" : "충청북도 충주시",
            "changeto" : "교현.안림동"
             }
        },
        {
             "안림동" : {
            "city" : "충청북도 충주시",
            "changeto" : "교현.안림동"
             }
        },
        {
             "목행동" : {
            "city" : "충청북도 충주시",
            "changeto" : "목행.용탄동"
             }
        },
        {
             "용탄동" : {
            "city" : "충청북도 충주시",
            "changeto" : "목행.용탄동"
             }
        },
        {
             "성내동" : {
            "city" : "충청북도 충주시",
            "changeto" : "성내.충인동"
             }
        },
        {
             "충인동" : {
            "city" : "충청북도 충주시",
            "changeto" : "성내.충인동"
             }
        },
        {
             "칠금동" : {
            "city" : "충청북도 충주시",
            "changeto" : "칠금.금릉동"
             }
        },
        {
             "금릉동" : {
            "city" : "충청북도 충주시",
            "changeto" : "칠금.금릉동"
             }
        },
        {
             "호암동" : {
            "city" : "충청북도 충주시",
            "changeto" : "호암.직동"
             }
        },
        {
             "직동" : {
            "city" : "충청북도 충주시",
            "changeto" : "호암.직동"
             }
        },
    ]
}